## Part 3: Data Analytics

In [12]:
# Imports and Class Definition

import gzip
import io
import json

import boto3
import pandas as pd


class S3DataLoader:
    def __init__(self, bucket_name):
        self.bucket_name = bucket_name
        self.s3_client = boto3.client('s3')

    def load_csv(self, key):
        csv_obj = self.s3_client.get_object(Bucket=self.bucket_name, Key=key)
        body_bytes = csv_obj['Body'].read()

        if body_bytes[:2] == b'\x1f\x8b':  # gzip magic bytes
            decompressed = gzip.decompress(body_bytes)
        else:
            decompressed = body_bytes

        df = pd.read_csv(io.BytesIO(decompressed), sep="\t", engine="python")
        df.columns = [col.strip() for col in df.columns]
        df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        return df

    def load_json(self, key, json_data_key=None):
        json_obj = self.s3_client.get_object(Bucket=self.bucket_name, Key=key)
        body = json_obj['Body'].read().decode('utf-8')
        data = json.loads(body)
        if json_data_key:
            df = pd.json_normalize(data[json_data_key])
        else:
            df = pd.json_normalize(data)
        return df


In [13]:
# Set variables and load data

bucket_name = "bls-data-sync-sri"
csv_key = 'bls/pr/pr.data.0.Current'  # Your CSV key
json_key = 'datausa/response.json'  # Your JSON key

loader = S3DataLoader(bucket_name)

df_csv = loader.load_csv(csv_key)
print("CSV DataFrame head:")
print(df_csv.head())

df_json = loader.load_json(json_key, json_data_key='data')
print("\nJSON DataFrame head:")
print(df_json.head())


CSV DataFrame head:
     series_id  year period  value footnote_codes
0  PRS30006011  1995    Q01    2.6            NaN
1  PRS30006011  1995    Q02    2.1            NaN
2  PRS30006011  1995    Q03    0.9            NaN
3  PRS30006011  1995    Q04    0.1            NaN
4  PRS30006011  1995    Q05    1.4            NaN

JSON DataFrame head:
  ID Nation         Nation  ID Year  Year  Population    Slug Nation
0   01000US  United States     2023  2023   332387540  united-states
1   01000US  United States     2022  2022   331097593  united-states
2   01000US  United States     2021  2021   329725481  united-states
3   01000US  United States     2020  2020   326569308  united-states
4   01000US  United States     2019  2019   324697795  united-states


C:\Users\csris_i58bcul\AppData\Local\Temp\ipykernel_26892\2500619906.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [14]:
# Step 1: Convert relevant columns
df_json['Year'] = pd.to_numeric(df_json['Year'], errors='coerce')
df_json['Population'] = pd.to_numeric(df_json['Population'], errors='coerce')

# Filter the years between 2013 and 2018
filtered = df_json[(df_json['Year'] >= 2013) & (df_json['Year'] <= 2018)]

# Compute mean and standard deviation
mean_population = filtered['Population'].mean()
std_population = filtered['Population'].std()

#Display the results
print(f"📊 Mean population (2013–2018): {mean_population:,.2f}")
print(f"📈 Standard deviation: {std_population:,.2f}")

📊 Mean population (2013–2018): 317,437,383.00
📈 Standard deviation: 4,257,089.54


In [15]:
# 🔢 Step 2: Convert value to float and year to int

df_csv['value'] = pd.to_numeric(df_csv['value'], errors='coerce')
df_csv['year'] = pd.to_numeric(df_csv['year'], errors='coerce')

#Group by series_id and year, then sum the values
grouped = df_csv.groupby(['series_id', 'year'])['value'].sum().reset_index()

# Find the best year per series_id (max sum of value)
best_years = grouped.loc[grouped.groupby('series_id')['value'].idxmax()]
best_years = best_years.sort_values(by='series_id').reset_index(drop=True)

print("Best year per series_id with summed value:")
print(best_years.head(10))  # or to_csv, to_excel, etc.


Best year per series_id with summed value:
     series_id  year    value
0  PRS30006011  2022   20.500
1  PRS30006012  2022   17.100
2  PRS30006013  1998  705.895
3  PRS30006021  2010   17.700
4  PRS30006022  2010   12.400
5  PRS30006023  2014  503.216
6  PRS30006031  2022   20.500
7  PRS30006032  2021   17.100
8  PRS30006033  1998  702.672
9  PRS30006061  2022   37.000


In [16]:
# Step 3: Filter Part 1 DataFrame for the specific series_id and period
filtered_csv = df_csv[
    (df_csv['series_id'] == 'PRS30006032') &
    (df_csv['period'] == 'Q01')
    ].copy()

# Ensure the year column is numeric for join
filtered_csv['year'] = pd.to_numeric(filtered_csv['year'], errors='coerce')

# Prepare Part 2 DataFrame (population) for merge
df_json['Year'] = pd.to_numeric(df_json['Year'], errors='coerce')
df_json['Population'] = pd.to_numeric(df_json['Population'], errors='coerce')
df_population = df_json[['Year', 'Population']].rename(columns={'Year': 'year'})

# Merge on year
merged_df = pd.merge(filtered_csv, df_population, on='year', how='left')

# Final display
print("🔗 Joined Report:")
print(merged_df[['series_id', 'year', 'period', 'value', 'Population']])


🔗 Joined Report:
      series_id  year period  value   Population
0   PRS30006032  1995    Q01    0.0          NaN
1   PRS30006032  1996    Q01   -4.2          NaN
2   PRS30006032  1997    Q01    2.8          NaN
3   PRS30006032  1998    Q01    0.9          NaN
4   PRS30006032  1999    Q01   -4.1          NaN
5   PRS30006032  2000    Q01    0.5          NaN
6   PRS30006032  2001    Q01   -6.3          NaN
7   PRS30006032  2002    Q01   -6.6          NaN
8   PRS30006032  2003    Q01   -5.7          NaN
9   PRS30006032  2004    Q01    2.0          NaN
10  PRS30006032  2005    Q01   -0.5          NaN
11  PRS30006032  2006    Q01    1.8          NaN
12  PRS30006032  2007    Q01   -0.8          NaN
13  PRS30006032  2008    Q01   -3.5          NaN
14  PRS30006032  2009    Q01  -21.0          NaN
15  PRS30006032  2010    Q01    3.2          NaN
16  PRS30006032  2011    Q01    1.5          NaN
17  PRS30006032  2012    Q01    2.5          NaN
18  PRS30006032  2013    Q01    0.5  311536594.0
19 

In [17]:
# Step 4: Filter merged_df to only rows where Population is available
final_report = merged_df[merged_df['Population'].notna()].copy()

# Optional: sort by year if you like
final_report = final_report.sort_values(by='year')

# Final Output
print("📊 Final Report (Filtered with Available Population):")
print(final_report[['series_id', 'year', 'period', 'value', 'Population']])


📊 Final Report (Filtered with Available Population):
      series_id  year period  value   Population
18  PRS30006032  2013    Q01    0.5  311536594.0
19  PRS30006032  2014    Q01   -0.1  314107084.0
20  PRS30006032  2015    Q01   -1.7  316515021.0
21  PRS30006032  2016    Q01   -1.4  318558162.0
22  PRS30006032  2017    Q01    0.9  321004407.0
23  PRS30006032  2018    Q01    0.5  322903030.0
24  PRS30006032  2019    Q01   -1.6  324697795.0
25  PRS30006032  2020    Q01   -7.0  326569308.0
26  PRS30006032  2021    Q01    0.7  329725481.0
27  PRS30006032  2022    Q01    5.3  331097593.0
28  PRS30006032  2023    Q01    0.3  332387540.0


### 📌 Observations

The value in Part 1 represents quarterly measurements for various indicators. Summing these quarterly values by year helps identify the best year with the highest total value for each series_id.

The population data (Part 2) provides yearly US population counts, and calculating mean and standard deviation for 2013–2018 shows population trends to compare against economic indicators.

- 📈 **Highest Value:** `5.3%` in **2022**, with a population of over **331M**.
- 📉 **Lowest Value:** `-7.0%` in **2020**, coinciding with the first pandemic year.
- 🧮 **Overall Trend:** The population increases steadily each year, while the `value` fluctuates.
- 🧭 **Recent Values (last 3 years):**
  - **2021:** `0.7%`
  - **2022:** `5.3%`
  - **2023:** `0.3%`
